Bevor Sie dieses Problem einreichen, stellen Sie sicher, dass alles wie erwartet läuft. Starten Sie zuerst **den Kernel neu** (wählen Sie in der Menüleiste Kernel$\rightarrow$Restart) und führen Sie dann **jede Zelle aus** (wählen Sie in der Menüleiste Cell$\rightarrow$Run Cells ausführen). Wenn Sie Fragen zum Jupyterhub oder zur Aufgabe haben, stehe ich Ihnen gerne zur Verfügung: quang.tran@fh-dortmund.de

Stellen Sie sicher, dass Sie alle Stellen ausfüllen, an denen "YOUR CODE HERE" oder "YOUR ANSWER HERE" steht, sowie Ihren Namen und Ihre Mitarbeiter unten:


In [ ]:
NAME = ""
Mitarbeiter = ""

---

<img src="fhlogo.png" width="350">

# Lokalisierung mit erweitertem Kalman-Filter (5.3 von Hertzberg et al.)

In dieser Übung werden wir einen erweiterten Kalman-Filter verwenden, um die Ego-Position des Roboters zu schätzen.

---
## Teil A: Theoretisch (erweitertem) Kalman Filter. Abk. (E)KF (3 Punkte)

Bitte füllen Sie alle untenstehenden Antworten aus:

Was ist der Unterschied zwischen KF und EKF?

YOUR ANSWER HERE

ist der Kalman-Filter ein Bayes-Filter? warum ist er das/nicht?

YOUR ANSWER HERE

Können Sie einige Anwendungen des (erweiterten) Kalman-Filters auflisten?

YOUR ANSWER HERE

---

## Teil B: Filter Design (3 Punkte)

Stellen Sie sich eine Situation vor, in der wir einen Odometrie, einen Gyroskop und ein GPS haben. Wir möchten die Position des Roboters/Auto mithilfe von EKF schätzen.


In dieser Simulation hat der Roboter einen Zustandsvektor von der Zustandsraumdarstellung wie folgt:



$$\textbf{x}_t=[x_t, y_t, \phi_t, v_t]^T$$
wo:

x, y sind die 2D x-y ego-position, 

$\phi$ ist ego-orientatierung,

v ist ego-geschwindigkeit.

Dank der Odometrie (Geschwindigkeitssensor) und des Gyroskops können wir die lineare und die Winkelgeschwindigkeit des Fahrzeugs steuern. So i.d.R kann der Eingangsvetor als beliebiger Zeitschritt verwendet werden.

$$\textbf{u}_t=[v_t, \omega_t]$$

Außerdem verfügt der Roboter über einen GNSS-Sensor, d. h. der Roboter kann die x-y-Position zu jedem Zeitpunkt beobachten. Beachten Sie, dass diese Sensoren immer Fehler haben, die wir später besprechen werden.

$$\textbf{z}_t=[x_t,y_t]$$


Zunächst importieren wir die benötigte Python-Bibliothek, indem wir die folgenden Zellen wie math, ploting, numpy und scipy ausführen.
### Simulationsparameter

Hier haben wir die Simulationsparameter:

Eingang ($v, w$) Geräusch / GPS Geräusch

Zeittick: DT

lineare/angulare Geschwindigkeit

Simulationszeit/Rendering

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as Rot

#Simulationsparameter
INPUT_NOISE = np.diag([1.0, np.deg2rad(30.0)]) ** 2
GPS_NOISE = np.diag([0.5, 0.5]) ** 2 # diagonal matrix 0.5
DT = 0.1  # time tick [s]
SIM_TIME = 40.0  # simulation time [s]
show_animation = True

### Motion Model (Zustandsdifferenzialgleichung) 

The robot model is:

$$\dot{x} = vcos(\phi)$$

$$ \dot{y} = vsin((\phi)$$

$$ \dot{\phi} = \omega$$


So, the motion model is

$$\textbf{x}_{t+1} = A\textbf{x}_t+B\textbf{u}_t$$

wo:

$\begin{equation*}
A=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
B=
\begin{bmatrix}
cos(\phi)dt & 0\\
sin(\phi)dt & 0\\
0 & dt\\
1 & 0\\
\end{bmatrix}
\end{equation*}$

$dt$ ist ein Zeitintervall.

In [ ]:
A = np.array([[1.0, 0, 0, 0],
                  [0, 1.0, 0, 0],
                  [0, 0, 1.0, 0],
                  [0, 0, 0, 0]])
print (A)

In [ ]:
def motion_model(x, u):
    '''
    Aufgabe:
        Erstellen einer A- und B-Matrix als Python-Numpy-Array und die nextstate Gleichung
    Hinweis: 
        Verwenden Sie np.array() und die mathematische Bibliothek. 
        Verwenden Sie @ Infix-Operator für Matrixmultiplikation verwenden (z.B.: A @ x)
    Ergebnis: 
        x: nächsterZustand x (t+1)
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return x # nextstate

#Übergangsmatrizen 
def jacob_f(x, u):
    """
    Jacobian of Motion Model

    motion model
    x_{t+1} = x_t+v*dt*cos(yaw)
    y_{t+1} = y_t+v*dt*sin(yaw)
    yaw_{t+1} = yaw_t+omega*dt
    v_{t+1} = v{t}
    so
    dx/dyaw = -v*dt*sin(yaw)
    dx/dv = dt*cos(yaw)
    dy/dyaw = v*dt*cos(yaw)
    dy/dv = dt*sin(yaw)
    """
    yaw = x[2, 0]
    v = u[0, 0]
    jF = np.array([
        [1.0, 0.0, -DT * v * math.sin(yaw), DT * math.cos(yaw)],
        [0.0, 1.0, DT * v * math.cos(yaw), DT * math.sin(yaw)],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]])

    return jF

### Observation Model (Ausgangsgleichung)

Der Roboter kann x-y-Positionsinformationen vom GPS erhalten.

Das GPS-Observationmodelle ist also:

$$\textbf{z}_{t} = C\textbf{x}_t$$

wo:

$\begin{equation*}
C=
\begin{bmatrix}
1 & 0 & 0& 0\\
0 & 1 & 0& 0\\
\end{bmatrix}
\end{equation*}$

Jacobi-Matrix ist:

$\begin{equation*}
J_H=
\begin{bmatrix}
\frac{dx}{dx}& \frac{dx}{dy} & \frac{dx}{d\phi} &  \frac{dx}{dv}\\
\frac{dy}{dx}& \frac{dy}{dy} & \frac{dy}{d\phi} &  \frac{dy}{dv}\\
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
　=
\begin{bmatrix}
1& 0 & 0 & 0\\
0 & 1 & 0 & 0\\
\end{bmatrix}
\end{equation*}$

In [ ]:
def observation_model(x):
    '''
    Aufgabe:
        Erstellen einer C-Matrix als Python-Numpy-Array und die Output Gleichung z
    Hinweis: 
        Verwenden Sie np.array() und die mathematische Bibliothek. 
        Verwenden Sie @ Infix-Operator für Matrixmultiplikation verwenden (z.B.: A @ x)
    Ergebnis: 
        z: Output function
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return z

def jacob_h():
    # Jacobian of Observation Model
    jH = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0]
    ])

    return jH

### Extented Kalman Filter Algorithm

Lokalisierungsverfahren mit erweitertem Kalman-Filter:

=== Prädiktion===

Motion command: $x_{Pred} = Ax_t+Bu_t$

Predicted covariance: $P_{Pred} = J_FP_t J_F^T + Q$

=== Korrektur ===

Observation: $z_{Pred} = Hx_{Pred}$ 

$y = z - z_{Pred}$

$S = J_H P_{Pred}.J_H^T + R$

Compute the Kalman gain: $K = P_{Pred}.J_H^T S^{-1}$

Estimated next state: $x_{t+1} = x_{Pred} + Ky$

Update the error covariance: $P_{t+1} = ( I - K J_H) P_{Pred}$

Denkfrage: Wenn wir KF verwenden, welcher Teil wird dann verändert?

In [ ]:
def ekf_estimation(xEst, PEst, z, u):
   
    '''
    Aufgabe:
        Prädiktion 
    Hinweis: 
        motion_model, jacob_f Funktion von oben verwenden
        Verwenden Sie @ Infix-Operator für Matrixmultiplikation verwenden 
    Ergebnis: 
        xPred: Predicted state
        PPred: Predicted covariance
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    
    #  Korrektur
    jH = jacob_h() #linearized
    zPred = observation_model(xPred)
    y = z - zPred
    S = jH @ PPred @ jH.T + R
    K = PPred @ jH.T @ np.linalg.inv(S)
    xEst = xPred + K @ y
    PEst = (np.eye(len(xEst)) - K @ jH) @ PPred
    return xEst, PEst

### EKF Kovarianzmatrix

$Q$: Prozessrauschen

$R$: Messrauschen

In [ ]:
'''
Aufgabe:
    Ändern Sie die Q- und R-Matrix, um den Unterschied zu sehen
Ergebnis: 
    Q: Prozessrauschen Matrix
    𝑅: Messrauschen Matrix
 '''
Q = np.diag([
    0.1,  # variance of location on x-axis
    0.1,  # variance of location on y-axis
    np.deg2rad(1.0),  # variance of yaw angle
    1.0  # variance of velocity
]) ** 2  # predict state covariance

R = np.diag([1.0, 1.0]) ** 2  # Observation x,y position covariance

'''
Nach der Änderung schreiben Sie (ohne Einrückung): 
    print (Q) 
    print (R)     
in den Block unten, um die Matrix zu sehen

'''
# YOUR CODE HERE
raise NotImplementedError()



### Hilfsmittel Funktion für die Simulation

Eingabe berechnen

Messung/Prozess mit Rauschen generieren 

Kovarianz-Eclipse aufzeichnen


In [ ]:
def calc_input():
    v = 1.0  # [m/s]
    yawrate = 0.1  # [rad/s]
    u = np.array([[v], [yawrate]])
    return u

def observation(xTrue, xd, u): #generates the input and observation vector with noises
    xTrue = motion_model(xTrue, u)

    # add noise to gps x-y
    z = observation_model(xTrue) + GPS_NOISE @ np.random.randn(2, 1)

    # add noise to input - reflect odom and gyro noise
    ud = u + INPUT_NOISE @ np.random.randn(2, 1)

    xd = motion_model(xd, ud)

    return xTrue, z, xd, ud

def plot_covariance_ellipse(xEst, PEst):  
    Pxy = PEst[0:2, 0:2]
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * math.pi + 0.1, 0.1)
    a = math.sqrt(eigval[bigind])
    b = math.sqrt(eigval[smallind])
    x = [a * math.cos(it) for it in t]
    y = [b * math.sin(it) for it in t]
    angle = math.atan2(eigvec[1, bigind], eigvec[0, bigind])
    rot = Rot.from_euler('z', angle).as_matrix()[0:2, 0:2]
    fx = rot @ (np.array([x, y]))
    px = np.array(fx[0, :] + xEst[0, 0]).flatten()
    py = np.array(fx[1, :] + xEst[1, 0]).flatten()
    plt.plot(px, py, "--r")

# Hauptprogramm

In [ ]:
if __name__ == '__main__':  
    time = 0.0

    # State Vector [x y yaw v]'
    xEst = np.zeros((4, 1))
    xTrue = np.zeros((4, 1))
    PEst = np.eye(4)

    xDR = np.zeros((4, 1))  # Dead reckoning

    #store history
    hxEst = xEst
    hxTrue = xTrue
    hxDR = xTrue
    hz = np.zeros((2, 1))

    while SIM_TIME >= time:
        time += DT
        u = calc_input()

        xTrue, z, xDR, ud = observation(xTrue, xDR, u) #add noises

        xEst, PEst = ekf_estimation(xEst, PEst, z, ud)

        # store data history
        hxEst = np.hstack((hxEst, xEst))
        hxDR = np.hstack((hxDR, xDR))
        hxTrue = np.hstack((hxTrue, xTrue))
        hz = np.hstack((hz, z))

        if show_animation:
            plt.clf()
            # for stopping simulation with the esc key.
            plt.gcf().canvas.mpl_connect('key_release_event',
                    lambda event: [exit(0) if event.key == 'escape' else None])
            plt.plot(hz[0, :], hz[1, :], ".g")
            plt.plot(hxTrue[0, :].flatten(),
                     hxTrue[1, :].flatten(), "-b")
            plt.plot(hxDR[0, :].flatten(),
                     hxDR[1, :].flatten(), "-k")
            plt.plot(hxEst[0, :].flatten(),
                     hxEst[1, :].flatten(), "-r")
            plot_covariance_ellipse(xEst, PEst)
            plt.axis("equal")
            plt.grid(True)
            plt.pause(0.001)




<span style='color:Blue'> Die blaue Linie ist die Bodenwahrheit.  </span>

Die schwarze Linie ist die Dead Reckoning Trajektorie.

<span style='color:Green'> Der grüne Punkt ist die Measurement (z. B. GPS).  </span>

<span style='color:Red'> Die rote Linie ist die geschätzte Trajektorie mit EKF.  </span>

<span style='color:Red'> Die rote Ellipse ist die geschätzte Kovarianz-Ellipse mit EKF.  </span>


Herzlichen Glückwunsch, Sie haben soeben einen erweiterten Kalman-Filter zur Verfolgung der Roboterposition entworfen.